In [2]:
from distributed import Client as DaskClient
import sqlalchemy as sa

from classify.features import *

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s - %(name)s.py]: %(message)s'
)

In [4]:
DaskClient()

distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-x4vglpxg', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-vt9jig9n', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-s3fze024', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-orjge57r', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57132,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:57145,Total threads: 1
Dashboard: http://127.0.0.1:57147/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:57137,


In [5]:
pd.set_option('max_columns', None)

In [6]:
config = ExpungeConfig.from_yaml('classify/expunge_config.yaml')

config.run_id

[INFO - classify.config.py]: Loading config from file: classify/expunge_config.yaml


'feature_testing'

In [7]:
query = (
    sa.select(expunge)
    .where(
        sa.and_(
            expunge.c.HearingDate < config.cutoff_date,
            # sa.or_(
            #     expunge.c.person_id == 1021000000606,
            #     expunge.c.person_id == 1070000000994,
            #     expunge.c.person_id == 163010000000348
            # )
        )
    )
    .order_by(
        expunge.c.person_id, 
        expunge.c.HearingDate
    )
)

In [8]:
%%time
ddf = fetch_expunge_data(
    config,
    custom_query=query,
    # n_partitions=3
)

ddf.head()

[INFO - classify.features.py]: Reading from table: expunge_clean


CPU times: user 490 ms, sys: 191 ms, total: 680 ms
Wall time: 3.52 s


,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195


In [9]:
ddf

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
npartitions=1,,,,,,,,,
1.001000e+12,datetime64[ns],object,object,object,object,object,object,object,int64
3.662210e+14,...,...,...,...,...,...,...,...,...


In [10]:
ddf = clean_data(ddf)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195


In [11]:
%%time
ddf = (
    ddf.pipe(build_disposition)
       .pipe(build_chargetype)
       .pipe(build_codesection, config)
       .pipe(build_convictions)
       .pipe(build_date_features)
       .pipe(build_timedelta_features, config)
       .pipe(build_class_features)
)

ddf.head()

CPU times: user 1.04 s, sys: 108 ms, total: 1.14 s
Wall time: 6.35 s


,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,date_if_conviction,date_if_felony_conviction,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_after_misdemeanor,next_conviction_disqualifier_after_felony,pending_after_misdemeanor,pending_after_felony,is_class_1_or_2,is_class_3_or_4,class1_2,class3_4
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,False,False


In [12]:
ddf = remove_unneeded_columns(ddf)
ddf = append_run_id(ddf, config)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_after_misdemeanor,next_conviction_disqualifier_after_felony,pending_after_misdemeanor,pending_after_felony,class1_2,class3_4,run_id
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,feature_testing
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,feature_testing
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,feature_testing
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,feature_testing
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaN,NaN,614.0,4556,False,False,True,True,False,False,False,False,feature_testing


In [13]:
%%time
file_paths = write_to_csv(ddf)

[INFO - classify.features.py]: Expungement feature data will be written to: /tmp/expunge_data
[INFO - classify.features.py]: Clearing any data from previous runs
[INFO - classify.features.py]: Command 'rm -rf /tmp/expunge_data/expunge_features-*.csv' returned with exit value: 0
[INFO - classify.features.py]: Executing Dask task graph and writing results to CSV...
[INFO - classify.features.py]: File(s) written successfully


CPU times: user 432 ms, sys: 80.9 ms, total: 513 ms
Wall time: 4.61 s


In [14]:
file_paths[:5]

['/tmp/expunge_data/expunge_features-0.csv']

In [15]:
%%time
load_to_db(file_paths)

[INFO - classify.features.py]: Opening connection to PostGres via Psycopg
[INFO - classify.features.py]: Loading from file: /tmp/expunge_data/expunge_features-0.csv
[INFO - classify.features.py]: Load to DB complete


CPU times: user 109 ms, sys: 40.1 ms, total: 150 ms
Wall time: 1.01 s
